In [191]:
import pandas as pd
import numpy as np
import re
import datetime as dt
import os

PATH = r'C:\Users\danie\Dropbox\Documents\projects\bls_api_project'
NOW = dt.datetime.now().strftime('%d-%b-%Y_%H:%M:%S')

nr = pd.read_csv(os.path.normpath(f'{PATH}/outputs/main_op/national_results.csv'))
sr = pd.read_csv(f'{PATH}/outputs/main_op/state_results.csv')
ssd = pd.read_csv(f'{PATH}/outputs/state_scrape_op/state_series_dimension.csv')
st = pd.read_csv(f'{PATH}/outputs/excel_op/survey_table.csv')
nsd= pd.read_csv(f'{PATH}/outputs/excel_op/national_series_dimension_og.csv')

In [ ]:
# Drop duplicates for national_series_dimension 

def drop(df):
    df = df.drop_duplicates(subset='seriesID',keep='first', ignore_index=True)
    df = df.dropna(axis=0)

    return df

nsd = drop(nsd)
ssd = drop(ssd)



In [ ]:
# Replacing '-' with None and converting to float

def values_to_floats(df):
    df['value'] = df['value'].replace('-', None)
    df['value'] = df['value'].astype('float')

    return df

values_to_floats(sr)
values_to_floats(nr)

,seriesID,year,period,period_name,value,footnotes
0,WSU100,2021,M12,December,1.0,[{}]
1,WSU100,2021,M11,November,1.0,[{}]
2,WSU100,2021,M10,October,4.0,[{}]
3,WSU100,2021,M09,September,1.0,[{}]
4,WSU100,2021,M08,August,1.0,[{}]
...,...,...,...,...,...,...
38950,LEU0252919700,2003,Q01,1st Quarter,1104.0,[{}]
38951,LEU0252919700,2002,Q04,4th Quarter,1125.0,[{}]
38952,LEU0252919700,2002,Q03,3rd Quarter,1101.0,[{}]
38953,LEU0252919700,2002,Q02,2nd Quarter,1105.0,[{}]


In [194]:
# Remove spaces 

def space_remover(df):

        for col in df.columns:

                df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)
                
        return df


nr = space_remover(nr)
nsd = space_remover(nsd)
sr = space_remover(sr)
ssd = space_remover(ssd)
st = space_remover(st)              

In [196]:
def convert_adjusted(series):

    adjusted = ', seasonally adjusted'
    not_adjusted = ', not seasonally adjusted'

    def adjusted_column(text):
        text = str(text).lower()
        return True if text.endswith(adjusted) else False if text.endswith(not_adjusted) else None
    
    def remove_terms(text):
        text_lower = text.lower()
        index1 = text_lower.find(adjusted)
        index2 = text_lower.find(not_adjusted)
        if index1 != -1:
            return text[:index1]
        elif index2 != -1:
            return text[:index2]
        return text
    
    
    series = series.copy()
    
    series['is_adjusted'] = series['series'].apply(adjusted_column)
    series['series'] = series['series'].apply(remove_terms)

    return series

nsd = convert_adjusted(nsd)
ssd = convert_adjusted(ssd)

In [ ]:
# Reorder columns

ssd = ssd[['seriesID', 'series', 'state','survey','is_adjusted']]
nsd = nsd[['seriesID', 'series', 'survey', 'is_adjusted']]

In [309]:
# Save to CSV in cleaned folder

nsd.to_csv(f'{PATH}outputs/cleaning_op/national_series_dimension_cleaned_{NOW}.csv', index=False)
ssd.to_csv(f'{PATH}outputs/cleaning_op/state_series_dimension_cleaned_{NOW}.csv', index=False)
sr.to_csv(f'{PATH}outputs/cleaning_op/state_results_cleaned_{NOW}.csv', index=False)
nr.to_csv(f'{PATH}outputs/cleaning_op/national_results_cleaned_{NOW}.csv', index=False)